# Architecture & Design Patterns

Good software architecture is about making your code easy to understand, change, and test. This notebook covers the key architectural patterns used in this blueprint to achieve loose coupling, robustness, and maintainability.

## 1. Dependency Inversion with `typing.Protocol`

**The Problem:** Your core business logic becomes tightly coupled to external systems like databases, APIs, or file systems. If your `user_service` directly imports and uses a `PostgresClient`, it's impossible to test the service without a real database, and switching to a different database becomes a major refactoring effort.

**The Solution:** We use `typing.Protocol` to invert the dependency. Instead of depending on a *concrete* class, our service depends on an *abstract interface* (the protocol). Any class that has the methods and signatures defined in the protocol can be used, without needing to inherit from it. This is also known as "Static Duck Typing".

This allows us to easily swap implementations, for example, using a real database client in production and a simple mock object in our tests.

In [ ]:
from typing import Protocol, TypeVar

from returns.result import Failure, Result, Success

# --- 1. Define the Abstract Contract ---
KeyType = TypeVar("KeyType")
ReturnType = TypeVar("ReturnType")


class Fetcher(Protocol[KeyType, ReturnType]):
    """A generic contract for any object that can fetch data by a key."""

    def fetch_by_id(self, key: KeyType) -> ReturnType | None: ...


# --- 2. Define our business model and logic ---
class User:
    def __init__(self, name: str):
        self.name = name


# This service depends on the ABSTRACT Fetcher, not a concrete one.
def get_user_name(user_fetcher: Fetcher[int, User], user_id: int) -> Result[str, str]:
    maybe_user = user_fetcher.fetch_by_id(user_id)
    if maybe_user is None:
        return Failure(f"User {user_id} not found.")
    return Success(maybe_user.name)


# --- 3. Create Concrete Implementations ---


# A mock implementation for testing. It fulfills the Fetcher protocol.
class MockUserFetcher:
    def fetch_by_id(self, key: int) -> User | None:
        if key == 1:
            return User(name="Mock Alice")
        return None


# A "real" implementation. It also fulfills the protocol.
class RealDatabaseFetcher:
    def fetch_by_id(self, key: int) -> User | None:
        # In a real app, this would connect to a DB.
        print(f"\n(Connecting to DB to fetch user {key}...)")
        db_data = {1: "Real Bob", 2: "Real Charlie"}
        name = db_data.get(key)
        return User(name=name) if name else None


# --- Verification ---
mock_fetcher = MockUserFetcher()
real_fetcher = RealDatabaseFetcher()

# The SAME service function works with the MOCK fetcher:
mock_result = get_user_name(mock_fetcher, 1)
assert mock_result.unwrap() == "Mock Alice"
print("✅ Service function works correctly with the mock fetcher.")

# The SAME service function works with the REAL fetcher:
real_result = get_user_name(real_fetcher, 2)
assert real_result.unwrap() == "Real Charlie"
print("✅ Service function works correctly with the real fetcher.")

# The function also handles failures from any fetcher:
not_found_result = get_user_name(real_fetcher, 99)
assert isinstance(not_found_result, Failure)
print("✅ Service function handles failures correctly.")

## 2. Exhaustive Checks with Pattern Matching

**The Problem:** When you have a function that accepts a `Union` of different types, you often use `if/isinstance` checks to handle each type. It's easy to forget to handle a new type if the `Union` is ever expanded, leading to runtime errors.

**The Solution:** Python's `match/case` statement (PEP 634) is a powerful tool for structural pattern matching. When used with a strict type checker like `BasedPyright`, it can perform **exhaustiveness checking**. If you have a `Union[A, B]` and only write a `case` for `A`, the type checker will raise an error, forcing you to handle all possible types.

**Caveat:** As noted in the project setup, mypy's plugin for the `returns` library currently has a bug that can lead to false positives when pattern matching on `Result`. For this reason, we recommend using pattern matching primarily on your own `Union` types or Pydantic models, and using `.map`/`.lash` for `Result`.

In [ ]:
class Circle:
    def __init__(self, radius: float):
        self.radius = radius


class Rectangle:
    def __init__(self, width: float, height: float):
        self.width = width
        self.height = height


Shape = Circle | Rectangle


def get_area(shape: Shape) -> float:
    match shape:
        case Circle(radius=r):
            # Inside this block, the type checker KNOWS `shape` is a Circle.
            return 3.14 * r * r
        case Rectangle(width=w, height=h):
            # And here, it KNOWS `shape` is a Rectangle.
            return w * h

    # If we added a `Square` to the `Shape` Union but forgot to add a `case` here,
    # a strict type checker would raise an error, saving us from a runtime bug.


# --- Verification ---
circle = Circle(10)
rectangle = Rectangle(5, 10)

assert get_area(circle) == 314.0
assert get_area(rectangle) == 50.0
print("✅ Pattern matching correctly calculates area for all shapes.")

## 3. Best Practices for Decorators

**The Problem:** Decorators are a powerful feature, but a naive implementation can have unintended side effects: the decorated function loses its original name and docstring, and type hints for the arguments and return value are broken.

**The Solution:** We use `functools.wraps` to preserve the original function's metadata and `typing.ParamSpec` and `typing.TypeVar` to correctly type the decorator, preserving the original function's signature.

In [ ]:
import functools
import time
from collections.abc import Callable
from typing import ParamSpec, TypeVar

P = ParamSpec("P")
T = TypeVar("T")


def timing_decorator[P, T](func: Callable[P, T]) -> Callable[P, T]:
    """A well-behaved decorator that logs function execution time."""

    @functools.wraps(func)  # <-- Preserves metadata like __name__ and __doc__
    def wrapper(*args: P.args, **kwargs: P.kwargs) -> T:
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value

    return wrapper


@timing_decorator
def example_function(name: str) -> str:
    """This is a simple example function."""
    time.sleep(0.01)
    return f"Hello, {name}"


# --- Verification ---
result = example_function("World")

# 1. Verify the function still works
assert result == "Hello, World"

# 2. Verify metadata is preserved thanks to @wraps
assert example_function.__name__ == "example_function"
assert "simple example function" in example_function.__doc__

print("✅ Decorator works and preserves function metadata.")